In [1]:
import pandas as pd
import numpy as np
import glob

/var/folders/jn/5hx9ymq526v78x75qss7g_ww0000gn/T/ipykernel_20524/2720773279.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Merge all csv files
merged_csv = glob.glob("/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/*.csv")
merged_csv

["/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['THD'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['DAD'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['BMV'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['HAN'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['DLI'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['PQC'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['VII'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['UIH'].csv",
 "/Users/luongtricuong/Desktop/Cuong_Luong/Ticket-Price-Analysis-and-Prediction/raw_data/flights_['VCS'].csv",
 

In [3]:
df = pd.concat([pd.read_csv(file) for file in merged_csv ], ignore_index=True)
df

,Origin_Airport,Destination_Airport,Departure_Time,Arrival_Time,Airline,Duration,Date,Price
0,THD,SGN,7:30 PM,9:30 PM,VietJet Air,2 hr,2024-06-26,"₫1,783,800"
1,THD,SGN,12:55 PM,2:55 PM,VietJet Air,2 hr,2024-06-26,"₫1,956,600"
2,THD,SGN,3:20 PM,5:20 PM,VietJet Air,2 hr,2024-06-26,"₫1,956,600"
3,THD,SGN,7:30 PM,9:30 PM,VietJet Air,2 hr,2024-06-27,"₫1,783,800"
4,THD,SGN,12:55 PM,2:55 PM,VietJet Air,2 hr,2024-06-27,"₫1,956,600"
...,...,...,...,...,...,...,...,...
11979,VDO,SGN,8:55 AM,11:05 AM,Vietnam Airlines,2 hr 10 min,2024-07-06,"₫1,940,000"
11980,VDO,SGN,9:50 AM,12:05 PM,VietJet Air,2 hr 15 min,2024-07-07,"₫2,084,600"
11981,VDO,SGN,9:50 AM,12:05 PM,VietJet Air,2 hr 15 min,2024-07-10,"₫2,084,600"
11982,VDO,SGN,9:50 AM,12:05 PM,VietJet Air,2 hr 15 min,2024-07-12,"₫2,084,600"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11984 entries, 0 to 11983
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Origin_Airport       11984 non-null  object
 1   Destination_Airport  11984 non-null  object
 2   Departure_Time       11808 non-null  object
 3   Arrival_Time         11808 non-null  object
 4   Airline              11626 non-null  object
 5   Duration             11808 non-null  object
 6   Date                 11984 non-null  object
 7   Price                11808 non-null  object
dtypes: object(8)
memory usage: 749.1+ KB


In [5]:
df.dtypes

Origin_Airport         object
Destination_Airport    object
Departure_Time         object
Arrival_Time           object
Airline                object
Duration               object
Date                   object
Price                  object
dtype: object

In [6]:
df.isnull().sum()

Origin_Airport           0
Destination_Airport      0
Departure_Time         176
Arrival_Time           176
Airline                358
Duration               176
Date                     0
Price                  176
dtype: int64

## Cleaning Web Scraped Data
For flight data that price is not included, we will remove it from our dataframe

In [7]:
df.dropna(inplace=True)

In [8]:
def convert_to_min(duration):
    element = duration.split()
    if len(element) == 2:
        if element[1] == 'hr':
            return int(element[0]) * 60
        elif element[1] == 'min':
            return int(element[0])
    elif len(element) == 4:
        return int(element[0]) * 60 + int(element[2])

df['Duration'] = df['Duration'].apply(convert_to_min)

In [9]:
df = df.rename(columns={'Duration': 'duration_min', 
                        'Origin_Airport': 'origin',
                        'Destination_Airport': 'destination',
                        'Departure_Time': 'depart_time',
                        'Arrival_Time': 'arrive_time',
                        'Airline': 'airliner',
                        'Price': 'price',
                        'Date': 'date'})

In [10]:
df['price'] = df['price'].str.replace('₫', '')
df['price'] = df['price'].str.replace(',', '')

In [11]:
df['depart_time'] = pd.to_datetime(df['depart_time'], format = '%I:%M %p')
df['depart_time'] = df['depart_time'].dt.time

df['arrive_time'] = df['arrive_time'].str.replace('+1', '')
df['arrive_time'] = pd.to_datetime(df['arrive_time'], format = '%I:%M %p')
df['arrive_time'] = df['arrive_time'].dt.time

df['date'] = pd.to_datetime(df['date'])
df['price'] = df['price'].astype(int)

In [12]:
df[['origin', 'destination', 'airliner']] = df[['origin', 'destination', 'airliner']].astype('string')

In [13]:
def convert_airliner_to_icao_code(airline_name):
    list_airlines = {
        'Operated by Vietnam Air Service': 'Vietnam Airlines'}
    for key, value in list_airlines.items():
        if key in airline_name:
            return value
    return airline_name

df['airliner'] = df['airliner'].apply(convert_airliner_to_icao_code)
df

,origin,destination,depart_time,arrive_time,airliner,duration_min,date,price
0,THD,SGN,19:30:00,21:30:00,VietJet Air,120,2024-06-26,1783800
1,THD,SGN,12:55:00,14:55:00,VietJet Air,120,2024-06-26,1956600
2,THD,SGN,15:20:00,17:20:00,VietJet Air,120,2024-06-26,1956600
3,THD,SGN,19:30:00,21:30:00,VietJet Air,120,2024-06-27,1783800
4,THD,SGN,12:55:00,14:55:00,VietJet Air,120,2024-06-27,1956600
...,...,...,...,...,...,...,...,...
11979,VDO,SGN,08:55:00,11:05:00,Vietnam Airlines,130,2024-07-06,1940000
11980,VDO,SGN,09:50:00,12:05:00,VietJet Air,135,2024-07-07,2084600
11981,VDO,SGN,09:50:00,12:05:00,VietJet Air,135,2024-07-10,2084600
11982,VDO,SGN,09:50:00,12:05:00,VietJet Air,135,2024-07-12,2084600


In [14]:
def iata_to_airport_name(iata):
    list_airports = {
        'HAN': 'Noi Bai Intl Airport',
        'SGN': 'Tan Son Nhat Intl Airport',
        'HUI': 'Phu Bai Airport',
        'CXR': 'Cam Ranh Intl Airport',
        'PQC': 'Phu Quoc Intl Airport',
        'PXU': 'Pleiku Airport',
        'UIH': 'Phu Cat Airpot',
        'VKG': 'Rach Gia Airport',
        'VCL': 'Chu Lai Airport',
        'THD': 'Tho Xuan Airport',
        'TBB': 'Tuy Hoa Airport',
        'VII': 'Vinh Airport',
        'BMV': 'Buon Ma Thuot Airport',
        'CAH': 'Ca Mau Airport',
        'VCA': 'Can Tho Intl Airport',
        'VCS': 'Con Dao Airport',
        'DLI': 'Lien Khuong Intl Airport',
        'DAD': 'Da Nang Intl Airport',
        'DIN': 'Dien Bien Airport',
        'VDH': 'Dong Hoi Airport',
        'VDO': 'Van Don Airport',
        'HPH': 'Cat Bi Intl Airport'}
    for key, value in list_airports.items():
        if key in iata:
            return value
    return iata

df['airport_origin_name'] = df['origin'].apply(iata_to_airport_name)
df['airport_destination_name'] = df['destination'].apply(iata_to_airport_name)

In [15]:
df.iloc[:, [0,8,1,9,2,3,4,5,6,7]]

,origin,airport_origin_name,destination,airport_destination_name,depart_time,arrive_time,airliner,duration_min,date,price
0,THD,Tho Xuan Airport,SGN,Tan Son Nhat Intl Airport,19:30:00,21:30:00,VietJet Air,120,2024-06-26,1783800
1,THD,Tho Xuan Airport,SGN,Tan Son Nhat Intl Airport,12:55:00,14:55:00,VietJet Air,120,2024-06-26,1956600
2,THD,Tho Xuan Airport,SGN,Tan Son Nhat Intl Airport,15:20:00,17:20:00,VietJet Air,120,2024-06-26,1956600
3,THD,Tho Xuan Airport,SGN,Tan Son Nhat Intl Airport,19:30:00,21:30:00,VietJet Air,120,2024-06-27,1783800
4,THD,Tho Xuan Airport,SGN,Tan Son Nhat Intl Airport,12:55:00,14:55:00,VietJet Air,120,2024-06-27,1956600
...,...,...,...,...,...,...,...,...,...,...
11979,VDO,Van Don Airport,SGN,Tan Son Nhat Intl Airport,08:55:00,11:05:00,Vietnam Airlines,130,2024-07-06,1940000
11980,VDO,Van Don Airport,SGN,Tan Son Nhat Intl Airport,09:50:00,12:05:00,VietJet Air,135,2024-07-07,2084600
11981,VDO,Van Don Airport,SGN,Tan Son Nhat Intl Airport,09:50:00,12:05:00,VietJet Air,135,2024-07-10,2084600
11982,VDO,Van Don Airport,SGN,Tan Son Nhat Intl Airport,09:50:00,12:05:00,VietJet Air,135,2024-07-12,2084600


In [16]:
df.to_csv('flights.csv', index=False)